# Exploratory notebook for IMACLIM within MATMat-Trade

## Creating the MatMat-Trade model base on IMACLIM regions and sectors but on EXIOBASE 3 data

In [1]:
# make the necessary imports
import tqdm
import shutil
import warnings
import pandas as pd
import numpy as np
import multiprocessing


from main import Model
from src.utils import footprint_extractor
from src.scenarios import DICT_SCENARIOS
from src.advance import extract_data

In [2]:
# for some reason saved model has issues with the names of regions so we rebuild the model each times


shutil.rmtree('data/models/2015__pxp__IMACLIM_format__ghg')

In [3]:


model = Model(aggregation_name="IMACLIM_format")

#ignore performace warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Loading data... (may take a few minutes)
Data loaded successfully !


## Creating the scenarios based on IMACLIM scenarios and the projected changes 

In [4]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

for scenario in tqdm.tqdm(scenario_list[:1]):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario="INDC",new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario="INDC",new_name="IMACLIM_all_cons_change_2050_"+scenario)

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

## Getting the emissions per scenario 

### Single process version, might be long to run 

In [ ]:

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


for scenario in tqdm.tqdm(scenario_list):
    # create 2050 version of each scenarios applying emissivity changes and then technological changes
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    Emissions.loc[(scenario),"2050"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Prod"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    Emissions.loc[(scenario),"2050-Cons"]=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Cons"]=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050-Prod"]=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    production_based_emissions.loc[(scenario),"2050"]=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    

### Multiple Process versio, should be faster

In [4]:

NBR_PROCESS=4

scenario_list=["INDC","INDC2020_1000","INDC2030_1000","INDC2020_1600","INDC2020_400","INDC2030_1600","baseline"]

start_year=pd.concat([ pd.concat([pd.DataFrame(footprint_extractor(model,region=region),index=[2015]).T  for region in model.regions],
                      keys=model.regions,
                      names=("regions","types")) for scenario in scenario_list],
          keys=scenario_list,
          names=("scenario","regions","types"))

Emissions=pd.concat([start_year for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]).sort_index()
Emissions.columns=["2015","2050","2050-Prod","2050-Cons"]

production_based_emissions=pd. concat([ pd.concat([ model.iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"]for i in range(4)],axis=1,keys=["2015","2050","2050-Prod","2050-Cons"]) for scenario in scenario_list],
                                        keys=scenario_list,
                                        names=("scenario","region","sector"),
                                        axis=0).sort_index()


def task(scenario):
    model.new_counterfactual("IMACLIM_all_2050_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=2050,scenario=scenario)
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=2050,scenario=scenario)
    # create the 2050 version based on production changes observed in IMACLIM (since we work on groos output, we need a reference to apply relative change : x_ref), with name new_name
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["production_change_IMACLIM"], year=2050,scenario=scenario,new_name="IMACLIM_all_prod_change_2050_"+scenario,x_ref=model.iot.x.sort_index())
    
    model.modify_counterfactual("IMACLIM_all_2050_"+scenario,DICT_SCENARIOS["consumption_change_imaclim"], year=2050,scenario=scenario,new_name="IMACLIM_all_cons_change_2050_"+scenario)
    
    ret1=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret2=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret3=pd.concat([pd.DataFrame(footprint_extractor(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario],region=region),index=[2050]).T  for region in model.regions],
                        keys=model.regions,
                        names=("regions","types")).sort_index().values
    
    ret4=model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret5=model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    ret6=model.counterfactuals["IMACLIM_all_2050_"+scenario].iot.stressor_extension.D_pba.sum(axis=0,level=1).loc["CO2"].sort_index().values
    
    del(model.counterfactuals["IMACLIM_all_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_cons_change_2050_"+scenario])
    del(model.counterfactuals["IMACLIM_all_prod_change_2050_"+scenario])
    
    return ret1,ret2,ret3,ret4,ret5,ret6


pool = multiprocessing.Pool(processes=NBR_PROCESS)
result_list=pool.map(task,scenario_list)

for i in range(len(scenario_list)):
    scenario=scenario_list[i]
    Emissions.loc[(scenario),"2050"],Emissions.loc[(scenario),"2050-Prod"],Emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Cons"],production_based_emissions.loc[(scenario),"2050-Prod"],production_based_emissions.loc[(scenario),"2050"]=result_list[i]
        
    

Problems on sums of columns  region  sector               
AFR     Terrestrial transport     333.209566
MDE     Terrestrial transport    4486.348277
RAL     Terrestrial transport     172.151602
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     47.043673
MDE     Terrestrial transport    146.430499
RAL     Terrestrial transport     25.875496
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     237.323465
MDE     Terrestrial transport    3148.991362
RAL     Terrestrial transport     124.123202
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     169.042045
MDE     Terrestrial transport    2024.383010
RAL     Terrestrial transport      89.719931
dtype: float64
Problems on sums of columns  region  sector               
AFR     Terrestrial transport     282.256477
MDE     Terrestrial transport    6422.740342
RAL     Terrestria

## Exploring the results

In [5]:
#adding IMACLIM direct emissions to the df
IMACLIM_emissions=extract_data(aggregation="IMACLIM_format")[6]

production_based_emissions["2050-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2050].rename_axis(index=str.lower)*1e9
production_based_emissions["2015-IMACLIM"]=IMACLIM_emissions.drop("World",level=1)[2015].rename_axis(index=str.lower)*1e9

In [6]:
diff_rapport =((production_based_emissions["2050-IMACLIM"]/production_based_emissions["2015-IMACLIM"])-(production_based_emissions["2050-Prod"]/production_based_emissions["2015"]))

for scenario in scenario_list:
    print(scenario,diff_rapport.loc[scenario].max())

INDC 8.268941087408166e-12
INDC2020_1000 8.035572207631958e-12
INDC2030_1000 7.099210108663101e-12
INDC2020_1600 8.144818153255073e-12
INDC2020_400 6.13709083552294e-12
INDC2030_1600 8.04800670550776e-12
baseline 9.425571434462654e-12


In [7]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region","sector"]).sum().loc["baseline"] 
Relative_error_2015=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015=Relative_error_2015*100
pd.concat([Relative_error_2015.loc[region] for region in Relative_error_2015.index.get_level_values("region").unique()],keys=Relative_error_2015.index.get_level_values("region").unique(),axis=1)

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
sector,,,,,,,,,,,,
Agriculture,-43.590339,37.134146,15.936325,-44.977011,-79.050224,-7.480474,48.257791,-60.567556,-55.777698,-18.200986,30.372903,-3.244593
Air,-45.142647,-22.186058,12.029989,71.750897,-2.496690,22.254563,46.981273,-44.709538,47.393486,-67.742879,39.151737,-7.363344
Coal,37.069602,43.351854,-88.326434,4.551393,-52.544245,-51.589768,96.477647,-35.200343,100.000000,100.000000,99.602004,-67.145125
Construction,84.523051,99.707779,76.583065,96.611540,71.457156,94.403003,96.048193,72.736079,83.043970,98.905166,96.741961,89.447613
Elec,-39.063432,72.813771,-61.749840,38.125084,-96.819867,-56.260718,27.300270,-32.720686,16.044896,-21.715810,5.708682,-39.344545
Gas,50.168897,81.877660,8.406563,97.880386,4.636883,87.340254,68.934163,100.000000,-13.784912,-2.699496,49.584495,71.617184
Industry,-43.194885,5.480878,-51.361436,21.771290,-72.690944,-30.088759,-9.858828,-60.712734,38.940292,9.302386,11.266783,-54.420385
Liquid fuels,-91.607445,-76.416717,-38.014303,-44.130341,-55.153795,-77.602159,-2.684537,-82.068727,100.000000,100.000000,-61.019781,99.869409
Maritime,-89.620191,-69.921130,46.180283,70.196181,100.000000,41.037293,93.972451,40.129431,-24.836034,-70.236938,-23.554738,-42.914006


In [8]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region","sector"]).sum().loc["baseline"] 
Relative_error_2015=(Relative_error_2015["2050-Prod"]-Relative_error_2015["2050-IMACLIM"])/np.max([Relative_error_2015["2050-Prod"],Relative_error_2015["2050-IMACLIM"]],axis=0)
Relative_error_2015=Relative_error_2015*100
pd.concat([Relative_error_2015.loc[region] for region in Relative_error_2015.index.get_level_values("region").unique()],keys=Relative_error_2015.index.get_level_values("region").unique(),axis=1)

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
sector,,,,,,,,,,,,
Agriculture,-43.590339,37.134146,15.936325,-44.977011,-79.050224,-7.480474,48.257791,-60.567556,-55.777698,-18.200986,30.372903,-3.244593
Air,-45.142647,-22.186058,12.029989,71.750897,-2.496690,22.254563,46.981273,-44.709538,47.393486,-67.742879,39.151737,-7.363344
Coal,37.069602,43.351854,-88.326434,4.551393,-52.544245,-51.589768,96.477647,-35.200343,100.000000,100.000000,99.602004,-67.145125
Construction,84.523051,99.707779,76.583065,96.611540,71.457156,94.403003,96.048193,72.736079,83.043970,98.905166,96.741961,89.447613
Elec,-39.063432,72.813771,-61.749840,38.125084,-96.819867,-56.260718,27.300270,-32.720686,16.044896,-21.715810,5.708682,-39.344545
Gas,50.168897,81.877660,8.406563,97.880386,4.636883,87.340254,68.934163,100.000000,-13.784912,-2.699496,49.584495,71.617184
Industry,-43.194885,5.480878,-51.361436,21.771290,-72.690944,-30.088759,-9.858828,-60.712734,38.940292,9.302386,11.266783,-54.420385
Liquid fuels,-91.607445,-76.416717,-38.014303,-44.130341,-55.153795,-77.602159,-2.684537,-82.068727,100.000000,100.000000,-61.019781,99.869409
Maritime,-89.620191,-69.921130,46.180283,70.196181,100.000000,41.037293,93.972451,40.129431,-24.836034,-70.236938,-23.554738,-42.914006


In [9]:
Relative_error_2015=production_based_emissions.groupby(["scenario","region"]).sum().loc["baseline"]
Relative_error_2015_ratio=(Relative_error_2015["2015"]-Relative_error_2015["2015-IMACLIM"])/np.max([Relative_error_2015["2015"],Relative_error_2015["2015-IMACLIM"]],axis=0)
Relative_error_2015_ratio=Relative_error_2015_ratio*100
pd.DataFrame(Relative_error_2015_ratio)

,0
region,
AFR,-41.880004
BRA,12.042277
CAN,-21.174507
CHN,42.804422
CIS,-57.997896
EUR,-16.077678
IND,26.399883
JAN,-38.587448
MDE,20.911331


In [10]:
Relative_error_2015["2015"]+model.iot.stressor_extension.F_Y.sum(axis=1,level=0).loc["CO2"]

region
AFR    1.075643e+12
BRA    4.973495e+11
CAN    5.678739e+11
CHN    9.929628e+12
CIS    1.475975e+12
EUR    4.548643e+12
IND    2.118380e+12
JAN    1.628660e+12
MDE    2.189486e+12
RAL    1.286805e+12
RAS    3.212389e+12
USA    5.145479e+12
dtype: float64

In [11]:
import src.advance
final_data_ratio,final_technical_coef,Link_country,Link,Production_volumes,total_consumption,direct_emissions=src.advance.extract_data("IMACLIM_format")

### Comparing Consumption emission

In [12]:
import src.advance
conso=src.advance.exctract_consumption_emission("IMACLIM_format")

/home/ubuntu/miniforge3/envs/MatMat-Trade-env/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [13]:
model_base_year_CO2_conso=model.iot.stressor_extension.F_Y.groupby("region",axis=1).sum().loc["CO2"]
IMACLIM_base_year_CO2_conso=conso.groupby(["Scenario","Region"]).sum().loc["baseline",2015]*1e9
comparison=pd.DataFrame([model_base_year_CO2_conso,IMACLIM_base_year_CO2_conso]).T
comparison.rename(columns={"CO2":"MRIO",2015:"IMACLIM"},inplace=True)
comparison.loc["World","MRIO"]=comparison.loc[:,"MRIO"].fillna(0).sum()
comparison

,MRIO,IMACLIM
AFR,3.122832e+11,3.058000e+11
BRA,8.498662e+10,1.121000e+11
CAN,1.266648e+11,1.297600e+11
CHN,4.574793e+11,5.429000e+11
CIS,2.479844e+11,5.006000e+11
EUR,9.358370e+11,1.360400e+12
IND,2.191543e+11,1.737400e+11
JAN,2.213855e+11,5.126000e+11
MDE,3.301295e+11,3.148000e+11
RAL,2.712905e+11,3.603000e+11


In [14]:
residential=conso.loc[("INDC",slice(None),"Residential"),2015]*1e9
residential.drop("World",level=1,inplace=True)

In [15]:
residential

Scenario  Region  Variable   
INDC      USA     Residential    3.680000e+11
          CAN     Residential    4.539000e+10
          EUR     Residential    5.168000e+11
          JAN     Residential    1.056000e+11
          CIS     Residential    3.436000e+11
          CHN     Residential    4.092000e+11
          IND     Residential    1.220000e+11
          BRA     Residential    2.538000e+10
          MDE     Residential    1.927000e+11
          AFR     Residential    1.121000e+11
          RAS     Residential    1.044000e+11
          RAL     Residential    1.046000e+11
Name: 2015, dtype: float64

In [16]:
imaclim_2=pd.read_excel("../Data/IMACLIM/IMACLIM2.0/IMACLIM_navigate_outputs_NAV_NPi_Default.xlsx",sheet_name="data",index_col=[1,2,3]).drop(columns=["Unit","Model"])

In [17]:
emissions=imaclim_2.loc[pd.Series(imaclim_2.index.get_level_values("Variable"),index=imaclim_2.index).apply(lambda x : "Emissions" in x)].dropna(axis=0)
emissions.reset_index(inplace=True)

In [18]:
emissions.loc[:,"Variable"]=emissions.loc[:,"Variable"].apply(lambda x : x.split('|')[-1])
emissions=emissions.loc[emissions.loc[:,"Variable"].isin(["Electricity","Transportation","Industry","Residential and Commercial","Other Sector"])]
emissions.set_index(["Scenario","Region","Variable"],inplace=True)

In [19]:
emissions

2015      2020  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1971.00   1668.00   
                       Industry                      506.30    556.80   
                       Residential and Commercial    495.90    526.00   
                       Transportation               1952.00   2058.00   
                       Other Sector                   25.85     24.73   
...                                                     ...       ...   
                World  Electricity                 11750.00  10470.00   
                       Industry                     8078.00   8718.00   
                       Residential and Commercial   2688.00   2912.00   
                       Transportation               7413.00   7635.00   
                       Other Sector                  138.10    146.60   

                                                       2025      2030  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1583.00   1528.00   
                       Industry                      638.00    692.70   
                       Residential and Commercial    558.00    574.50   
                       Transportation               2130.00   2119.00   
                       Other Sector                   26.92     29.08   
...                                                     ...       ...   
                World  Electricity                  9801.00   9563.00   
                       Industry                    10580.00  11930.00   
                       Residential and Commercial   3178.00   3313.00   
                       Transportation               8566.00   9252.00   
                       Other Sector                  177.60    201.10   

                                                       2035      2040  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1490.00   1345.00   
                       Industry                      718.90    717.00   
                       Residential and Commercial    578.50    568.10   
                       Transportation               1978.00   1790.00   
                       Other Sector                   30.95     32.12   
...                                                     ...       ...   
                World  Electricity                  9267.00   8967.00   
                       Industry                    12910.00  13690.00   
                       Residential and Commercial   3328.00   3268.00   
                       Transportation               9622.00   9763.00   
                       Other Sector                  218.50    233.60   

                                                       2045      2050  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1198.00   1099.00   
                       Industry                      716.90    715.10   
                       Residential and Commercial    552.80    539.10   
                       Transportation               1667.00   1631.00   
                       Other Sector                   33.28     34.67   
...                                                     ...       ...   
                World  Electricity                  9071.00   9434.00   
                       Industry                    14320.00  14930.00   
                       Residential and Commercial   3170.00   3105.00   
                       Transportation               9865.00  10050.00   
                       Other Sector                  248.30    261.70   

                                                       2055      2060  \
Scenario        Region Variable                                         
NAV_NPi_Default USA    Electricity                  1028.00    975.40   
                       Industry     

In [20]:
extract_data(aggregation="IMACLIM_format")[0]

2015      2016      2017      2018  \
scenario regions sector                                                      
INDC     USA     Agriculture             0.0 -0.029611 -0.060239 -0.092704   
                 Air                     0.0 -0.010650 -0.020562 -0.030603   
                 Coal                    0.0  0.000137  0.000396 -0.000015   
                 Construction            0.0 -0.017117 -0.034386 -0.051674   
                 Elec                    0.0 -0.049795 -0.090254 -0.124152   
...                                      ...       ...       ...       ...   
baseline World   Liquid fuels            0.0 -0.003790 -0.008628 -0.013676   
                 Maritime                0.0  0.000305 -0.000472  0.000296   
                 Oil                     0.0 -0.004719 -0.008595 -0.013010   
                 Services                0.0 -0.021620 -0.042577 -0.063089   
                 Terrestrial transport   0.0 -0.009181 -0.017205 -0.023572   

                                            2019      2020      2021  \
scenario regions sector                                                
INDC     USA     Agriculture           -0.124632 -0.158114 -0.191090   
                 Air                   -0.040548 -0.050891 -0.060721   
                 Coal                   0.000392  0.000178  0.000388   
                 Construction          -0.067387 -0.084396 -0.100210   
                 Elec                  -0.154610 -0.185410 -0.213301   
...                                          ...       ...       ...   
baseline World   Liquid fuels          -0.018713 -0.023445 -0.028415   
                 Maritime               0.001453  0.002218  0.003279   
                 Oil                   -0.016803 -0.020596 -0.023524   
                 Services              -0.083197 -0.102755 -0.122397   
                 Terrestrial transport -0.029324 -0.034501 -0.039907   

                                            2022      2023      2024  ...  \
scenario regions sector                                               ...   
INDC     USA     Agriculture           -0.226846 -0.261266 -0.296097  ...   
                 Air                   -0.070253 -0.080160 -0.089082  ...   
                 Coal                   0.000537  0.000424  0.000493  ...   
                 Construction          -0.116641 -0.131783 -0.147058  ...   
                 Elec                  -0.238976 -0.265000 -0.290214  ...   
...                                          ...       ...       ...  ...   
baseline World   Liquid fuels          -0.033392 -0.039900 -0.046829  ...   
                 Maritime               0.005038  0.004947  0.004871  ...   
                 Oil                   -0.025586 -0.028143 -0.030078  ...   
                 Services              -0.141338 -0.159605 -0.176947  ...   
                 Terrestrial transport -0.043707 -0.048624 -0.053708  ...   

                                            2041      2042      2043  \
scenario regions sector                                                
INDC     USA     Agriculture           -0.639190 -0.648047 -0.656219   
                 Air                   -0.264443 -0.275418 -0.285465   
                 Coal                   0.000317  0.000032  0.000269   
                 Construction          -0.387976 -0.401397 -0.414344   
                 Elec                  -0.730142 -0.752959 -0.777084   
...                                          ...       ...       ...   
baseline World   Liquid fuels          -0.004490  0.005242  0.017339   
                 Maritime              -0.029797 -0.033899 -0.038927   
                 Oil                   -0.066396 -0.068635 -0.070776   
                 Services              -0.387506 -0.396606 -0.405905   
                 Terrestrial transport -0.157975 -0.165041 -0.171128   

                                            2044      2045      2046  \
scenario regions sector                                                
INDC 

In [21]:
total_per_region=production_based_emissions.groupby(["scenario","region"]).sum().loc["baseline"] 
total_per_region["2015-IMACLIM2"]=emissions.groupby("Region").sum()[2015]*1e9
total_per_region["2050-IMACLIM2"]=emissions.groupby("Region").sum()[2050]*1e9
total_per_region.loc["World",:]=total_per_region.sum(axis=0)
total_per_region[["2015-IMACLIM2","2015","2015-IMACLIM"]]

,2015-IMACLIM2,2015,2015-IMACLIM
region,,,
AFR,1.014076e+12,7.633597e+11,1.313420e+12
BRA,4.820300e+11,4.123629e+11,3.627050e+11
CAN,4.454790e+11,4.412091e+11,5.597290e+11
CHN,9.255410e+12,9.472148e+12,5.417650e+12
CIS,1.939190e+12,1.227990e+12,2.923640e+12
EUR,3.348060e+12,3.612806e+12,4.304940e+12
IND,2.021034e+12,1.899226e+12,1.397832e+12
JAN,2.088313e+12,1.407275e+12,2.291510e+12
MDE,1.725960e+12,1.859356e+12,1.470540e+12


In [22]:
percentage_error=(total_per_region[["2015-IMACLIM2","2015-IMACLIM"]]/total_per_region[["2015"]].values-1)*100
percentage_error

,2015-IMACLIM2,2015-IMACLIM
region,,
AFR,32.843803,72.057822
BRA,16.894615,-12.042277
CAN,0.967762,26.862511
CHN,-2.288164,-42.804422
CIS,57.915741,138.083311
EUR,-7.327978,19.157809
IND,6.413587,-26.399883
JAN,48.394119,62.833162
MDE,-7.174318,-20.911331


In [23]:
percentage_error=(total_per_region[["2050-IMACLIM2","2050-IMACLIM"]]/total_per_region[["2050-Prod"]].values-1)*100
percentage_error

,2050-IMACLIM2,2050-IMACLIM
region,,
AFR,-25.482700,71.042086
BRA,7.859190,-0.915038
CAN,-46.362597,-21.793177
CHN,-31.254595,-43.734347
CIS,40.158633,110.796154
EUR,-9.870533,14.639856
IND,24.198937,-24.096440
JAN,34.308407,57.636492
MDE,-30.550991,-23.375312


In [24]:
iot=model.iot 
S = iot.stressor_extension.S
L = iot.L
Y_vect = iot.Y.sum(level=0, axis=1)
nbsectors = len(iot.get_sectors())

from pymrio.tools import ioutil
Y_diag = ioutil.diagonalize_blocks(Y_vect.values, blocksize=nbsectors)
Y_diag = pd.DataFrame(Y_diag, index=Y_vect.index, columns=Y_vect.index)
x_diag = L.dot(Y_diag)

dom_block = np.zeros((nbsectors, nbsectors))
x_trade = pd.DataFrame(
        ioutil.set_block(x_diag.values, dom_block),
        index=x_diag.index,
        columns=x_diag.columns,
    )

D_imp=model.iot.stressor_extension.D_imp

In [25]:
D_imp["EUR"].sum(axis=1)

region     
AFR     CO2    4.215630e+10
        CH4    1.448764e+11
        N2O    3.308509e+10
        SF6    1.884176e+08
        HFC    2.936723e+08
                   ...     
USA     CH4    2.903525e+10
        N2O    9.625195e+09
        SF6    8.757376e+08
        HFC    1.565615e+10
        PFC    2.096809e+08
Length: 72, dtype: float64

In [26]:
x_trade["EUR"].sum(axis=1).mul(S).groupby("region",axis=1).sum()

region,AFR,BRA,CAN,CHN,CIS,EUR,IND,JAN,MDE,RAL,RAS,USA
CO2,4.215630e+10,1.093705e+10,1.879780e+10,3.453716e+11,3.627702e+11,0.0,4.978231e+10,4.182936e+10,1.555254e+11,3.632721e+10,2.646223e+11,1.132833e+11
CH4,1.448764e+11,2.199508e+10,6.796729e+09,6.135159e+10,1.331055e+11,0.0,2.762634e+10,1.090417e+10,8.166520e+10,5.122717e+10,1.627362e+11,2.903525e+10
N2O,3.308509e+10,6.870553e+09,1.875541e+09,1.393661e+10,4.917610e+09,0.0,4.595196e+09,2.229424e+09,5.366147e+09,1.166579e+10,2.395465e+10,9.625195e+09
SF6,1.884176e+08,6.065500e+06,1.207730e+08,9.740243e+08,2.379570e+09,0.0,0.000000e+00,1.389078e+08,2.428405e+08,2.500373e+07,9.110735e+08,8.757376e+08
HFC,2.936723e+08,1.595827e+08,1.242788e+09,1.259597e+10,1.046155e+10,0.0,9.841514e+08,3.732939e+09,1.030510e+09,2.476998e+09,1.681112e+09,1.565615e+10
PFC,1.562639e+08,7.047311e+07,1.260338e+08,6.809228e+08,2.459993e+09,0.0,6.811150e+07,1.696673e+08,4.114282e+08,5.287699e+06,8.404008e+08,2.096809e+08


In [27]:
x_trade

region                               AFR                                     \
sector                       Agriculture         Air      Coal Construction   
region sector                                                                 
AFR    Agriculture              0.000000    0.000000  0.000000     0.000000   
       Air                      0.000000    0.000000  0.000000     0.000000   
       Coal                     0.000000    0.000000  0.000000     0.000000   
       Construction             0.000000    0.000000  0.000000     0.000000   
       Elec                     0.000000    0.000000  0.000000     0.000000   
...                                  ...         ...       ...          ...   
USA    Liquid fuels            89.060986   10.050314  0.396375    93.644819   
       Maritime                 1.173457    0.684385  0.028619     6.458294   
       Oil                     18.643007    8.711613  0.170410    27.413603   
       Services               407.501579  207.375165  8.259442  1836.869968   
       Terrestrial transport   57.655542  130.963832  2.382947   252.019147   

region                                                                         \
sector                              Elec        Gas     Industry Liquid fuels   
region sector                                                                   
AFR    Agriculture              0.000000   0.000000     0.000000     0.000000   
       Air                      0.000000   0.000000     0.000000     0.000000   
       Coal                     0.000000   0.000000     0.000000     0.000000   
       Construction             0.000000   0.000000     0.000000     0.000000   
       Elec                     0.000000   0.000000     0.000000     0.000000   
...                                  ...        ...          ...          ...   
USA    Liquid fuels            25.849938  11.959546   286.656177   138.657387   
       Maritime                 0.410872   0.277044    29.477614     0.345902   
       Oil                      5.491104   2.723281   110.405089    25.089755   
       Services               137.024789  98.586133  8070.470684   103.370738   
       Terrestrial transport   22.372351  15.892675  1174.270424    17.942331   

region                                              ...       USA  \
sector                          Maritime       Oil  ...      Coal   
region sector                                       ...             
AFR    Agriculture              0.000000  0.000000  ...  0.037229   
       Air                      0.000000  0.000000  ...  0.003474   
       Coal                     0.000000  0.000000  ...  0.120113   
       Construction             0.000000  0.000000  ...  0.002756   
       Elec                     0.000000  0.000000  ...  0.002601   
...                                  ...       ...  ...       ...   
USA    Liquid fuels            57.971559  1.742252  ...  0.000000   
       Maritime                 0.543215  0.017809  ...  0.000000   
       Oil                     12.227313  0.401227  ...  0.000000   
       Services               170.705725  5.202517  ...  0.000000   
       Terrestrial transport   65.617319  0.787258  ...  0.000000   

region                                                                        \
sector                       Construction       Elec        Gas     Industry   
region sector                                                                  
AFR    Agriculture             591.250035  16.946783  28.717114  3900.598459   
       Air                      23.354817   6.792597   3.144218   114.709277   
       Coal                     19.224029   7.232124   2.672172   219.965463   
       Construction             70.275122   1.621659   1.378356   110.852253   
       Elec                     19.163819   3.217212   1.344529   161.502853   
...                                   ...        ...        ...          ...   
USA    Liquid fuels              0.000000   0.000000   0.000000     0.0

In [28]:
model.iot.stressor_extension.S.shape[0]

6

In [29]:
np.array([[1,2] for i in range(10)]).flatten()

array([1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2])

In [30]:
stressors.flatten()

NameError: name 'stressors' is not defined

In [44]:
cop=pd.concat([x_trade.copy() for stressor in range(model.iot.stressor_extension.S.shape[0])])
cop.sort_index(inplace=True)
cop.reset_index(inplace=True)
stressors=np.array( [model.iot.stressor_extension.S.index.values for production_sources in x_trade.index]).flatten()
cop["stressor"]=stressors
cop.set_index(["region","sector","stressor"],inplace=True)
pd.concat([model.iot.stressor_extension.S.loc[stressor,(producing_region,producing_sector)]*x_trade.loc[(producing_region,producing_sector)]for producing_region,producing_sector,stressor in cop.index],
          keys=cop.index,
          axis=1).T.sort_index()

True

In [35]:
from src.utils import get_very_detailed_emissions
det=get_very_detailed_emissions(model.iot)

In [41]:

det_bis =det.copy()
for region in model.regions:
    det_bis.loc[region,region]=0

In [42]:
det_bis

region                                          AFR                \
sector                                  Agriculture           Air   
region sector                stressor                               
AFR    Agriculture           CH4       0.000000e+00  0.000000e+00   
                             CO2       0.000000e+00  0.000000e+00   
                             HFC       0.000000e+00  0.000000e+00   
                             N2O       0.000000e+00  0.000000e+00   
                             PFC       0.000000e+00  0.000000e+00   
...                                             ...           ...   
USA    Terrestrial transport CO2       7.127875e+06  1.619088e+07   
                             HFC       8.839170e+04  2.007806e+05   
                             N2O       6.720188e+04  1.526482e+05   
                             PFC       8.044089e+02  1.827205e+03   
                             SF6       4.307999e+03  9.785566e+03   

region                                                              \
sector                                          Coal  Construction   
region sector                stressor                                
AFR    Agriculture           CH4            0.000000  0.000000e+00   
                             CO2            0.000000  0.000000e+00   
                             HFC            0.000000  0.000000e+00   
                             N2O            0.000000  0.000000e+00   
                             PFC            0.000000  0.000000e+00   
...                                              ...           ...   
USA    Terrestrial transport CO2       294600.422672  3.115678e+07   
                             HFC         3653.295705  3.863705e+05   
                             N2O         2777.504346  2.937473e+05   
                             PFC           33.246827  3.516166e+03   
                             SF6          178.052854  1.883077e+04   

region                                                             \
sector                                         Elec           Gas   
region sector                stressor                               
AFR    Agriculture           CH4       0.000000e+00  0.000000e+00   
                             CO2       0.000000e+00  0.000000e+00   
                             HFC       0.000000e+00  0.000000e+00   
                             N2O       0.000000e+00  0.000000e+00   
                             PFC       0.000000e+00  0.000000e+00   
...                                             ...           ...   
USA    Terrestrial transport CO2       2.765863e+06  1.964789e+06   
                             HFC       3.429905e+04  2.436506e+04   
                             N2O       2.607666e+04  1.852411e+04   
                             PFC       3.121386e+02  2.217343e+02   
                             SF6       1.671653e+03  1.187494e+03   

region                                                             \
sector                                     Industry  Liquid fuels   
region sector                stressor                               
AFR    Agriculture           CH4       0.000000e+00  0.000000e+00   
                             CO2       0.000000e+00  0.000000e+00   
                             HFC       0.000000e+00  0.000000e+00   
                             N2O       0.000000e+00  0.000000e+00   
                             PFC       0.000000e+00  0.000000e+00   
...                                             ...           ...   
USA    Terrestrial transport CO2       1.451734e+08  2.218185e+06   
                             HFC       1.800274e+06  2.750739e+04   
                             N2O       1.368701e+06  2.091314e+04   
                             PFC       1.638340e+04  2.503310e+02   
                             SF6       8.774102e+04  1.340644e+03   

region                                                             ...  \
sector                                     M

In [138]:
model.iot.Y.sum(axis=1,level=0)

region                                  AFR           BRA           CAN  \
region sector                                                             
AFR    Agriculture            116424.660670  6.327438e+01    393.527651   
       Air                     12998.038157  8.879434e+01      0.241866   
       Coal                      672.849840  3.094753e-01      0.025423   
       Construction           218438.243355  3.814301e+01      0.000135   
       Elec                    22553.987906  1.867842e-08      0.007720   
...                                     ...           ...           ...   
USA    Liquid fuels              131.601618  2.826925e+02   1643.006900   
       Maritime                    0.000000  6.400854e+01      6.427084   
       Oil                         0.056273  5.431844e-03     29.234700   
       Services                 5082.453713  3.019560e+03   5044.503530   
       Terrestrial transport    2113.441198  2.566082e+03  33484.211154   

region                                 CHN          CIS           EUR  \
region sector                                                           
AFR    Agriculture              291.806539   876.250575  12348.094973   
       Air                        5.812041    60.454740    940.153024   
       Coal                       0.000002     0.093762     29.196547   
       Construction              86.833380    57.900888    409.076597   
       Elec                       0.556720     0.150681     11.679361   
...                                    ...          ...           ...   
USA    Liquid fuels               3.693454     0.213759   1778.474022   
       Maritime                  40.553367     2.811117    471.585889   
       Oil                        0.051662     0.039212     21.500021   
       Services               24298.955021  2396.510856  73107.895664   
       Terrestrial transport  23541.877483  1827.056974  29507.606155   

region                                IND           JAN           MDE  \
region sector                                                           
AFR    Agriculture            2918.380937    301.714321   1155.185472   
       Air                       1.226804    124.661431     23.768058   
       Coal                      5.670914      0.040226    112.644059   
       Construction              0.006038     17.918129     65.629565   
       Elec                      0.000000      0.795978      3.386936   
...                                   ...           ...           ...   
USA    Liquid fuels              8.291183     45.980128    243.558330   
       Maritime                  0.000000     88.240889      0.000000   
       Oil                       8.494133      0.364630     41.554232   
       Services               2148.196581  25302.558389   5047.038191   
       Terrestrial transport  1057.448015   6657.280066  15039.906365   

region                                 RAL           RAS           USA  
region sector                                                           
AFR    Agriculture               80.595248   3732.994483  4.835429e+02  
       Air                      152.630256    141.396744  1.550151e+02  
       Coal                       0.000791     57.871294  1.124686e-01  
       Construction              73.845113    159.789293  4.897279e+01  
       Elec                       0.739017      0.516745  1.575078e+00  
...                                    ...           ...           ...  
USA    Liquid fuels            9787.248648    506.387005  4.101635e+04  
       Maritime                   0.000000      8.397857  1.372821e+04  
       Oil                        0.501833     12.410045  1.981300e+04  
       Services               10962.196085   3034.695903  1.153094e+07  
       Terrestrial transport  35939.458363  13618.893114  7.959584e+05  

[144 rows x 12 columns]